In [6]:
# coding: utf-8

from  __future__ import absolute_import
from __future__ import print_function
from ImageDataGeneratorCustom import ImageDataGeneratorCustom
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.layers import *
from keras.models import Model, load_model
from keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator


In [7]:
def convnet_model_():
    model = tf.keras.Sequential([
        #VGG16
        tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3)),
        #GlobalAveragePooling2D
        tf.keras.layers.GlobalAveragePooling2D(),
        #Dense 4096 relu
        tf.keras.layers.Dense(4096, activation='relu'),
        #Dropout 0.5
        tf.keras.layers.Dropout(0.5),
        #Dense 4096 relu
        tf.keras.layers.Dense(4096, activation='relu'),
        #Dropout 0.5
        tf.keras.layers.Dropout(0.5),
        #L2 normalization
        tf.keras.layers.Lambda(lambda  x: K.l2_normalize(x,axis=1))
    ])

    return model

def deep_rank_model():
 
    convnet_model = convnet_model_()
    first_input = Input(shape=(224,224,3))
    first_conv = Conv2D(96, kernel_size=(8, 8),strides=(16,16), padding='same')(first_input)
    first_max = MaxPool2D(pool_size=(3,3),strides = (4,4),padding='same')(first_conv)
    first_max = Flatten()(first_max)
    first_max = Lambda(lambda  x: K.l2_normalize(x,axis=1))(first_max)

    second_input = Input(shape=(224,224,3))
    second_conv = Conv2D(96, kernel_size=(8, 8),strides=(32,32), padding='same')(second_input)
    second_max = MaxPool2D(pool_size=(7,7),strides = (2,2),padding='same')(second_conv)
    second_max = Flatten()(second_max)
    second_max = Lambda(lambda  x: K.l2_normalize(x,axis=1))(second_max)

    merge_one = concatenate([first_max, second_max])

    merge_two = concatenate([merge_one, convnet_model.output])
    emb = Dense(4096)(merge_two)
    l2_norm_final = Lambda(lambda  x: K.l2_normalize(x,axis=1))(emb)

    final_model = Model(inputs=[first_input, second_input, convnet_model.input], outputs=l2_norm_final)

    return final_model


In [8]:
deep_rank_model = deep_rank_model()

for layer in deep_rank_model.layers:
    print (layer.name, layer.output_shape)

model_path = "./deep_ranking"



vgg16_input [(None, 224, 224, 3)]
vgg16 (None, 7, 7, 512)
input_2 [(None, 224, 224, 3)]
input_3 [(None, 224, 224, 3)]
global_average_pooling2d (None, 512)
conv2d (None, 14, 14, 96)
conv2d_1 (None, 7, 7, 96)
dense (None, 4096)
max_pooling2d (None, 4, 4, 96)
max_pooling2d_1 (None, 4, 4, 96)
dropout (None, 4096)
flatten (None, 1536)
flatten_1 (None, 1536)
dense_1 (None, 4096)
lambda_1 (None, 1536)
lambda_2 (None, 1536)
dropout_1 (None, 4096)
concatenate (None, 3072)
lambda (None, 4096)
concatenate_1 (None, 7168)
dense_2 (None, 4096)
lambda_3 (None, 4096)


In [9]:
class DataGenerator(object):
    def __init__(self, params, target_size=(224, 224)):
        self.params = params
        self.target_size = target_size
        self.idg = ImageDataGeneratorCustom(**params)

    def get_train_generator(self, batch_size):
        return self.idg.flow_from_directory("./food_224/",
                                            batch_size=batch_size,
                                            target_size=self.target_size,shuffle=False,
                                            triplet_path  ='./train_triplets.txt'
                                           )

    def get_test_generator(self, batch_size):
        return self.idg.flow_from_directory("./food_224/",
                                            batch_size=batch_size,
                                            target_size=self.target_size, shuffle=False,
                                            triplet_path  ='./test_triplets.txt'
                                        )



dg = DataGenerator({
    "rescale": 1. / 255,
    "horizontal_flip": True,
    # "vertical_flip": True,
    "zoom_range": 0.2,
    "shear_range": 0.2,
    "rotation_range": 30,
    "fill_mode": 'nearest' 
}, target_size=(224, 224))



In [10]:
batch_size = 8
batch_size *= 3
train_generator = dg.get_train_generator(batch_size)



Found 10000 images belonging to 1 classes.


In [11]:
# train_generator.next()

In [12]:
_EPSILON = K.epsilon()
def _loss_tensor(y_true, y_pred):
    y_pred = K.clip(y_pred, _EPSILON, 1.0-_EPSILON)
    loss =  tf.convert_to_tensor(0,dtype=tf.float32)
    g = tf.constant(1.0, shape=[1], dtype=tf.float32)
    for i in range(0,batch_size,3):
        try:
            q_embedding = y_pred[i+0]
            p_embedding =  y_pred[i+1]
            n_embedding = y_pred[i+2]
            D_q_p =  K.sqrt(K.sum((q_embedding - p_embedding)**2))
            D_q_n = K.sqrt(K.sum((q_embedding - n_embedding)**2))
            loss = (loss + g + D_q_p - D_q_n )            
        except:
            continue
    loss = loss/(batch_size/3)
    zero = tf.constant(0.0, shape=[1], dtype=tf.float32)
    # return tf.maximum(loss,loss)
    return tf.maximum(loss,zero)



In [13]:
pred = deep_rank_model.predict((np.random.rand(10,224,224,3),np.random.rand(10,224,224,3),np.random.rand(10,224,224,3)))
print(pred.shape)
_loss_tensor(None,pred).numpy()

(10, 4096)


array([1.0008715], dtype=float32)

In [22]:
gen1 = ImageDataGenerator(rescale=1./255)
gen1 = gen1.flow_from_directory(batch_size=24,directory="./food_224/",shuffle=False,target_size=(224,224),class_mode=None)
gen2 = ImageDataGenerator(rescale=1./255)
gen2 = gen2.flow_from_directory(batch_size=24,directory="./food_224/",shuffle=False,target_size=(224,224),class_mode=None)
gen3 = ImageDataGenerator(rescale=1./255)
gen3 = gen3.flow_from_directory(batch_size=24,directory="./food_224/",shuffle=False,target_size=(224,224),class_mode=None)

class JoinedGen(tf.keras.utils.Sequence):
    def __init__(self, input_gen1, input_gen2, target_gen):
        self.gen1 = input_gen1
        self.gen2 = input_gen2
        self.gen3 = target_gen

        assert len(input_gen1) == len(input_gen2) == len(target_gen)

    def __len__(self):
        return len(self.gen1)

    def __getitem__(self, i):
        x1 = self.gen1[i]
        x2 = self.gen2[i]
        y = self.gen3[i]

        return [x1, x2, y], y

    def on_epoch_end(self):
        self.gen1.on_epoch_end()
        self.gen2.on_epoch_end()
        self.gen3.on_epoch_end()

my_gen = JoinedGen(gen1, gen2, gen3)


Found 10000 images belonging to 1 classes.
Found 10000 images belonging to 1 classes.
Found 10000 images belonging to 1 classes.


In [23]:
#deep_rank_model.load_weights('deepranking.h5')
deep_rank_model.compile(loss=_loss_tensor, optimizer='adam')


train_steps_per_epoch = int((59520)/batch_size)
train_epochs = 25
deep_rank_model.fit_generator(my_gen,
                        steps_per_epoch=train_steps_per_epoch,
                        epochs=train_epochs
                        )

model_path = "deepranking.h5"
deep_rank_model.save_weights(model_path)
#f = open('deepranking.json','w')
#f.write(deep_rank_model.to_json())
#f.close()

C:\Users\Timothe\AppData\Local\Temp/ipykernel_29844/1603943457.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  deep_rank_model.fit_generator(my_gen,


Epoch 1/25
    1/19840 [..............................] - ETA: 86:21:23 - loss: 0.9482

KeyboardInterrupt: 